In [ ]:
# Changing the working directory
import os
os.chdir('../')

In [1]:
from utils import *
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML

In [2]:
# Formating Pandas Dataframe
pd.options.display.float_format = '${:,.2f}'.format
pd.options.display.html.border = 2
pd.options.display.html.table_schema = True

In [3]:
# Reading the transactions from a Google Sheet
# data = readSpreadsheet(sheet_id='1c1Ou1VPT3qYzob267nnVdkKFZJX68KhMNRZ1XQMKwG0').convert_dtypes()
data = readSpreadsheet(sheet_id='1iqvTzSZ6COavTL1hDF3NU_9jZF-aEy06umGwhRbpCQQ').convert_dtypes()
data = data.astype({'TOTAL_AMT':'float'})

In [4]:
# Ccs50 Finance formating
def usd(value):
    """Format value as USD."""
    return f"${value:,.2f}"

In [5]:
# Loading the template 
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("pdf_report_templates/acountdetails.html")

In [6]:
# Finding the Running Balance for each student
data['Running Total'] = data.groupby('student_id').cumsum(axis=0)

In [7]:
# Finding each student's balance, filtering for peole who owe money
balances = data.groupby('student_id')['TOTAL_AMT'].sum().round(2)
balances.rename('Balance', inplace=True)
balances = balances[balances > 0].reset_index()

In [8]:
# Filtering data, renaming columns
clean_data = data.merge(balances, 'inner')[[
        'student_id', 'DESCRIPTION',
        'CHARGE_DATE', 'TOTAL_AMT', 
        'Running Total', 'Balance',
        'student_last', 'student_first'
    ]]

clean_data = clean_data.rename(columns={
        'DESCRIPTION':'Description',
        'CHARGE_DATE':'Date Charged',
        'TOTAL_AMT':'Charge Amount'
    })

In [9]:
# Pulling Students
students = clean_data.student_id.drop_duplicates().tolist()

In [10]:
for student in students:
    # Getting student's data
    df = clean_data[clean_data.student_id == student]
    balance = usd(df.Balance.drop_duplicates().values[0])
    name = df.student_first.drop_duplicates().values[0] + " " + df.student_last.drop_duplicates().values[0]
    df = df.drop(['student_last', 'student_first', 'student_id', 'Balance'], axis=1)
    # Printing the data as a pdf
    template_vars = {"title" : f"{name} Account Balance",
                 "total": balance,
                 "data": df.to_html(index=False, justify='left')
                }
    html_out = template.render(template_vars)
    HTML(string=html_out).write_pdf(f"finance/account_balances/{name}.pdf")

In [11]:
# Reseting pandas float formating 
pd.options.display.float_format = None

In [12]:
# Getting parents from the sky API 
parents = sky.getUsers('All School')[['id', 'email', 'phones', 'first_name', 'last_name', 'host_id']]

In [13]:
# Pulling parent wireless phone numbers
phones = parents.explode('phones')['phones'].apply(pd.Series).drop([
        0, 'links', 'type_id', 'id', 'index_id'], axis=1)

phones = phones[phones.phone_type == 'Wireless']

In [14]:
# Cleaning parents data
parents = parents.drop('phones', axis=1).rename(columns={'id':'user_id'})
parents = parents.merge(phones).drop('phone_type', axis=1)
parents['host_id'] = pd.to_numeric(parents.host_id, errors='coerce')

In [ ]:
contact_df = data[data.student_id.isin(students)][[
        'student_last', 'student_first',
        'parent1_host_id',
        'parent2_host_id'
    ]].drop_duplicates()

contact_df = contact_df.astype({
        'parent1_host_id':'float',
        'parent2_host_id':'float'
    })

In [ ]:
contact_df = contact_df.merge(
        parents.rename(columns={
            'host_id':'parent1_host_id',
            'user_id':'parent1_user_id',
            'email':'parent1_email',
            'first_name':'parent1_first',
            'last_name':'parent1_last',
            'number':'parent1_phone'
        }),
        'left'
    ).merge(
        parents.rename(columns={
            'host_id':'parent2_host_id',
            'user_id':'parent2_user_id',
            'email':'parent2_email',
            'first_name':'parent2_first',
            'last_name':'parent2_last',
            'number':'parent2_phone'
        }),
        'left'
    )

In [ ]:
# updateSpreadsheet(contact_df.fillna(""), sheet_id='1C3M0acCboR9wDJNBxRUAQbvFdZ4TmcYoXXNGcKSStdc')